In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# encoding: utf-8
import pymongo, json
import pandas as pd
from pandas import Series, DataFrame
from scipy.stats import pearsonr
import numpy as np

pd.set_option('display.width', None)  # 设置字符显示宽度
pd.set_option('display.max_rows', None)  # 设置显示最大行
pd.set_option('display.max_columns', None)  # 设置显示最大行

client = pymongo.MongoClient('localhost', 27017)
futures = client.futures
position = futures.position
market = futures.market
unit=futures.unit

start = '20190101'
end = '20200123'
var = 'MA'
days=30

market = DataFrame(list(market.find({'date': {'$gte': start}})))
position = DataFrame(list(position.find({'date': {'$gte': start}})))
unit= DataFrame(list(unit.find()))

# party_name = position[position['date'] >= end]
# party_name

party_name = position[position['variety'] == var]
long_party_name = party_name['long_party_name']
short_party_name = party_name['short_party_name']
party_name = long_party_name.append(short_party_name)
party_name = party_name.drop_duplicates(keep='first').dropna()
long = position.groupby(['date', 'variety', 'long_party_name'])[['long_openIntr']].sum()
short = position.groupby(['date', 'variety', 'short_party_name'])[['short_openIntr']].sum()
# 合并
frames = [long, short]
position = pd.concat(frames, axis=1, sort=True).fillna(0).reset_index()
# 净持仓
position['净持仓'] = position.apply(lambda x: x['long_openIntr'] - x['short_openIntr'], axis=1)
# 字段更名
position = position.rename(columns={'level_0': 'date', 'level_1': 'variety', 'level_2': 'mem'})
vars = position[position['variety'] == var]

df = pd.DataFrame()
for i in party_name:
    try:
        mem = vars[vars['mem'] == i]
        position_behind = mem.shift(-1)
        # # 合并滞后和原始数据
        all_position = pd.merge(position, position_behind, right_index=True, left_index=True)
        all_position = all_position[
            ['date_x', 'variety_x', 'mem_x', 'long_openIntr_x', 'short_openIntr_x', '净持仓_x', '净持仓_y']].dropna()

        #     更名
        all_position = all_position.rename(columns={'date_x': 'date', 'variety_x': 'variety', 'mem_x': '会员',
                                                    'long_openIntr_x': '多单量', 'short_openIntr_x': '空单量',
                                                    '净持仓_x': '当日净持仓', '净持仓_y': '昨日净持仓'})

        all_position['净持仓变化量'] = all_position.apply(lambda x: x['当日净持仓'] - x['昨日净持仓'], axis=1)

        market['change'] = market.apply(lambda x: x['close'] - x['open'], axis=1)
        #     print(market.info())
        vars1 = market[market['variety'] == var]
        # print(vars1.head(5))
        chg = vars1[['date', 'variety', 'change']]
        chg = chg.groupby(['date', 'variety'])['change'].sum()

        hb = pd.merge(chg, all_position, on=['date', 'variety'], how='outer').dropna().drop_duplicates()
        # print(hb)
        chgs = hb['change']
        # print(chgs)
        nets = hb['净持仓变化量']
        #
#         todayNetpostion = hb[hb['date'] <= end]
        # print(todayNetpostion)
        #
        # p = pearsonr(chgs, nets)[1]
        p=chgs.corr(nets)
        # r = pearsonr(chgs, nets)[1]

    #         p = pearsonr(todayNetpostion['change_y'],todayNetpostion['净持仓变化量'])[0]
    #         print(p)

        d=[{'var':var,'start':start,'end':end,'会员':i,'相关性':p,'样本':hb['净持仓变化量'].count(),'当日净持仓':hb['当日净持仓'].values[0],'昨日净持仓':hb['昨日净持仓'].values[0],'净持仓变化量':hb['净持仓变化量'].values[0]}]
        df1=pd.DataFrame(d)
        # print(df1)
        df=df.append(df1)
#         df== df.rolling(window=30)
        


    except:
        continue

df

""
